In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from time import time
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

% matplotlib inline

In [2]:
data = pd.read_csv('train.csv')

In [49]:
# data1 = data.iloc[::10]

In [68]:
relev_data = data.drop(columns=['index', 'ele_P', 'data_ind'])

In [69]:
relev_data = relev_data.dropna()

In [70]:
answers = relev_data.signal

In [71]:
relev_data = relev_data.drop(columns=['signal'])

In [72]:
# relev_data_one_hot = pd.get_dummies(relev_data, columns=["data_ind", ])

In [73]:
# relev_data_one_hot.head()

In [74]:
relev_data.head()

,TX,TY,X,Y,Z,chi2
0,0.013209,0.067202,66081.937500,52968.695312,11637.0,0.880506
1,0.014165,0.071018,66100.265625,53058.140625,12930.0,0.522405
2,0.017983,0.069956,66118.492188,53147.492188,14223.0,0.971824
3,0.013956,-0.058355,66121.281250,53082.601562,14223.0,0.771712
4,0.002209,0.045891,66116.429688,53143.609375,14223.0,0.345887


In [75]:
X_train, X_test, y_train, y_test = train_test_split(relev_data, answers, test_size = 0.2)

In [76]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)

In [77]:
ss = StandardScaler()
X_test_scaled = ss.fit_transform(X_test)

In [78]:
xgb_model = xgb.XGBClassifier()

In [79]:
y_train = np.array(y_train)

In [80]:
y_test = np.array(y_test)

In [81]:
xgb_model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [82]:
xg = xgb.DMatrix(X_test_scaled)

In [88]:
xg_prediction = xgb_model.predict_proba(X_test_scaled)

In [101]:
xg_prediction

(1499369, 2)

In [90]:
final_pred = xg_prediction[:, 1]

In [85]:
accuracy_score(xg_prediction, y_test)

0.9497782066989514

In [86]:
subm_sample = pd.read_csv("new_res_sample.csv")

In [100]:
len(final_pred)

1499369

In [107]:
final_test = pd.read_csv("new_test.csv")

In [108]:
final_test.head()

,index,TX,TY,X,Y,Z,chi2,ele_P,data_ind
0,0,-0.110714,-0.040569,59331.917969,49500.554688,6465.0,2.033096,NaN,221
1,1,-0.165904,0.159821,34738.796875,68337.164062,41376.0,0.369803,NaN,21
2,2,0.694244,-0.107642,68450.296875,45701.441406,59478.0,1.266626,NaN,347
3,3,0.370796,-0.016071,50257.199219,76170.953125,71115.0,2.767298,NaN,133
4,4,0.278292,0.332729,59803.960938,5938.398438,5172.0,2.794279,NaN,23


In [109]:
relev_final_test = final_test.drop(columns=['index', 'ele_P', 'data_ind'])

In [111]:
relev_final_test.head()

,TX,TY,X,Y,Z,chi2
0,-0.110714,-0.040569,59331.917969,49500.554688,6465.0,2.033096
1,-0.165904,0.159821,34738.796875,68337.164062,41376.0,0.369803
2,0.694244,-0.107642,68450.296875,45701.441406,59478.0,1.266626
3,0.370796,-0.016071,50257.199219,76170.953125,71115.0,2.767298
4,0.278292,0.332729,59803.960938,5938.398438,5172.0,2.794279


In [123]:
relev_final_test = relev_final_test.fillna(relev_final_test.mean())

In [124]:
ss = StandardScaler()
relev_final_test_scaled = ss.fit_transform(relev_final_test)

In [126]:
relev_final_test_scaled

array([[-0.42919805, -0.23893266,  0.79578217,  0.22851287, -1.47608791,
        -0.12682986],
       [-0.65177894,  0.56035862, -1.23364674,  1.17937315,  0.1446842 ,
        -2.52751005],
       [ 2.8172099 , -0.50646176,  1.54823248,  0.036736  ,  0.98508455,
        -1.23309881],
       ...,
       [ 0.07177776,  0.76405802,  1.52758052, -1.26688027,  1.04511315,
        -1.28337838],
       [ 1.45820098,  1.42204693, -0.0173308 ,  0.40025534, -0.57565896,
         0.54968936],
       [ 1.91581584, -1.71906449,  0.80565783, -1.3804845 , -1.71620229,
        -1.07273822]])

In [128]:
final_pred = xgb_model.predict_proba(relev_final_test_scaled)

In [130]:
final_pred[:, 1]

array([0.00283148, 0.7123721 , 0.23782459, ..., 0.15556279, 0.00358905,
       0.00950094], dtype=float32)

In [136]:
subm_sample.values[:, 1] = final_pred[:, 1]

In [140]:
subm_sample["signal"] = final_pred[:, 1]

In [141]:
subm_sample

,index,signal
0,0,0.002831
1,1,0.712372
2,2,0.237825
3,3,0.004335
4,4,0.000584
5,5,0.003994
6,6,0.001692
7,7,0.007371
8,8,0.006725
9,9,0.024241


In [142]:
subm_sample.to_csv("final_sumb.csv")

In [13]:
svc = SVC(verbose=True, probability=True)

In [14]:
X_train_scaled.shape

(5997472, 6)

In [ ]:
start = time() #00:37
svc.fit(X_train_scaled, np.array(y_train))
end = time()
print(end - start)

[LibSVM]

In [ ]:
ss = StandardScaler()

In [ ]:
X_test_scaled = ss.fit_transform(X_test)

In [ ]:
y_pred = svc.predict_proba(X_test_scaled)

In [ ]:
subm_sample = pd.read_csv('new_res_sample (1).csv')